### 本方案采用textrank4zh包实现问答摘要任务，取得了学习赛目前最高成绩，但得分仍然不高。
### 基于深度学习的文本生成模型（bart）等可能会取得更好的效果，期待paddle框架早日引入中文bart模型（认真脸）

In [1]:
# 导入pandas和textrank4zh
import pandas as pd
import numpy as np
!pip install textrank4zh
import codecs
from textrank4zh import TextRank4Sentence

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Preparing metadata (setup.py) ... done
  Created wheel for textrank4zh: filename=textrank4zh-0.3-py3-none-any.whl size=13419 sha256=92ef872b04259708c33a12d1978af9ab65912cfc2db906be99296a448fed0633
  Stored in directory: /home/aistudio/.cache/pip/wheels/af/c0/58/6ff06cd50dccfef4ee3bd37100a2ea0795ce7f80d8a97aecac
Successfully built textrank4zh

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# 数据读取与清洗

total = []

# 停用词表
replace_ls = ['技师说：', '[语音]', '[图片]', '你好', '您好','嗯','祝您生活愉快','祝您用车愉快','祝你用车愉快',
              '点击头像关注，以后可以在关注技师里发起图文咨询或者电话']

# 只加载测试集
with open('data/data1407/AutoMaster_TestSet.csv', 'r', encoding='UTF-8') as f:
    for i, line in enumerate(f):
        if i==0:
            continue
        line = line[:-1].split(',')

        # 留下顾客和技师的话
        texts = [line[-2]]
        # 只留下技师说的话
        # texts = []
        for text in line[-1].split('|'):
            if '技师说：' in text:
                for word in replace_ls:
                    text = text.replace(word, '')
                if text!='':
                    texts.append(text)
        total.append('%s\n' % ('|'.join(texts).replace('\t', '|')))

with open('data/test.tsv', 'w', encoding='UTF-8') as f:
    for line in total:
        f.write(line)

In [3]:
# 定义摘要函数，使用textrank工具包生成摘要
def summarize(text, num=4):    # num表示生成几句摘要，试验后取4句效果最好
    tr4s = TextRank4Sentence()
    tr4s.analyze(text=text, lower=True, source = 'all_filters')
    summary = ''
    for item in tr4s.get_key_sentences(num=num):
        summary = summary + item.sentence
    
    # 如果没有摘要，则填充无意义文字，避免影响csv读取
    if summary == '':
        summary = '祝您生活愉快'
    return summary

In [4]:
# 提取摘要
sums = []
from tqdm import tqdm
with open('data/test.tsv', 'r', encoding='UTF-8') as f:
    for line in tqdm(total):
        try:
            text = line.split('|')
            text.pop(0)
            text = '。'.join(text)
        except:
            text = line
        sums.append(summarize(text).strip('，'))

  0%|          | 0/20000 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.088 seconds.
Prefix dict has been built successfully.
100%|██████████| 20000/20000 [07:59<00:00, 41.72it/s]


In [5]:
# 生成预测结果文件
result = pd.DataFrame(columns=['QID','Prediction'])
result.QID = pd.Series(range(1,20001))
result.QID = result.QID.apply(lambda x:'Q'+str(x))
result.Prediction = sums

In [6]:
result[:20]

,QID,Prediction
0,Q1,如果已经超出了保修期建议你到当地的大型维修店进行检查，烧机油一般是发动机活塞环间隙过大和气门...
1,Q2,抛光处理一下就好了50元左右就好了，希望能够帮到你祝你生活愉快
2,Q3,气囊油丝坏了吗，这个价格不贵
3,Q4,一：发动机没力，并伴有“啪啪”的漏气声音二：发动机没力，并伴有排气管冒黑烟三：水温高，水箱盖...
4,Q5,希望能够帮到你可能前轮平衡快脱落或者不平衡造成的建议前轮做一下动平衡就好了
5,Q6,师傅，车子没有异常，那可能排气管关上缠到了方编袋造成的，或者车子开到修理厂清理一下，希望能够...
6,Q7,看看是不是绿色的哦这个是没有问题的，可以购买对的，如果要添加的话你还是需要添加绿色的防冻液的
7,Q8,如果机油灯常亮是不能再使用的，否则会对车辆发动机造成严重伤害，严重时会造成发动机抱死的故障解...
8,Q9,从这个图片来看，您的轮胎只是在胎面出现了一些裂痕，这种裂痕主要是由于行驶路面的问题导致的，但...
9,Q10,这种情况有可能是因为停放的时间久了之后，机油的润滑刚开始没有完全润滑到发动机的机械部件，所以...


In [7]:
# 保存预测结果文件并打包
result.to_csv("./result.csv", sep=',', index=False)
!zip 'submission.zip' 'result.csv'

updating: result.csv (deflated 64%)
